# Setting up an accessibility analysis Mozambique: Creating Destinations

### Downloading a Destinations from a polygon   

In [1]:
from pathlib import Path
import geopandas as gpd
import folium 
from shapely.geometry import LineString, Polygon, box
import rasterio
import matplotlib.pyplot as plt

In [2]:
import ra2ce.network.networks_utils as nut
from ra2ce.network.network_config_data.enums.network_type_enum import NetworkTypeEnum
from ra2ce.network.network_config_data.enums.road_type_enum import RoadTypeEnum
from ra2ce.network.network_config_data.network_config_data import (NetworkConfigData,NetworkSection,)
from ra2ce.network.network_wrappers.osm_network_wrapper.osm_network_wrapper import (OsmNetworkWrapper,)
from ra2ce.network.exporters.geodataframe_network_exporter import GeoDataFrameNetworkExporter
from ra2ce.network.exporters.multi_graph_network_exporter import MultiGraphNetworkExporter


In [36]:
root_dir = Path(r'P:\moonshot2-casestudy\RA2CE_HACKATHON_JULY\Automating_Destinations_and_buffer_creation')
Extent = root_dir/"Buffer_Polygons"/"buffer_polygon_OD.geojson"
Output_Destinations = root_dir.joinpath('static','network')
assert root_dir.exists()

### Determine the extent of the flood maps to download Destination

In [6]:
# Read the GeoJSON file
gdf = gpd.read_file(Extent)
gdf.explore()


In [19]:
import geopandas as gpd
from shapely.geometry import shape

OD_polygon = shape(gdf.geometry.iloc[0])
print(OD_polygon)

POLYGON ((39.138473831891424 -21.684747806909044, 32.3758853346633 -21.684747806909044, 32.24648880577371 -21.659009237016903, 32.13679172542509 -21.585711991285844, 32.063494479694036 -21.476014910937234, 32.037755909801895 -21.346618382047637, 32.037755909801895 -15.207378275768175, 32.063494479694036 -15.077981746878578, 32.13679172542509 -14.968284666529968, 32.24648880577371 -14.894987420798909, 32.3758853346633 -14.869248850906768, 39.138473831891424 -14.869248850906768, 39.26787036078102 -14.894987420798909, 39.377567441129635 -14.968284666529968, 39.45086468686069 -15.077981746878578, 39.47660325675283 -15.207378275768175, 39.47660325675283 -21.346618382047637, 39.45086468686069 -21.476014910937234, 39.377567441129635 -21.585711991285844, 39.26787036078102 -21.659009237016903, 39.138473831891424 -21.684747806909044))


### Downloading destinations data

In [20]:
import osmnx 
import pandas as pd

In [21]:
tags_basic_needs = {'amenity':['hospital','clinic','doctors','pharmacy']}  
features = osmnx.features_from_polygon(polygon=OD_polygon, tags=tags_basic_needs)


In [33]:
features['ID'] = range(len(features))
Destinations = features[["ID","amenity","name","geometry"]]
Destinations.rename(columns={"amenity": "category"}, inplace=True)
Destinations["geometry"] = Destinations.geometry.apply(lambda geom: geom.representative_point())

C:\Users\gunaratn\AppData\Local\Temp\ipykernel_6776\409382263.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Destinations.rename(columns={"amenity": "category"}, inplace=True)
c:\Users\gunaratn\AppData\Local\miniconda3\envs\ra2ce_env\lib\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [ ]:
#Add location
Destinations_clip = Destinations.clip(gdf_adm3)

In [37]:
Destinations.to_file(Output_Destinations/"Destinations.shp", driver='ESRI Shapefile')

C:\Users\gunaratn\AppData\Local\Temp\ipykernel_6776\2529720483.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  Destinations.to_file(Output_Destinations/"Destinations.shp", driver='ESRI Shapefile')
